<a href="https://colab.research.google.com/github/starryfieldz/Databusters/blob/main/NUS_DSESC_DATABUSTERS_32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) Libraries Installation
##### The cell below is to help you keep track the libraries used and install them quickly.
##### Ensure the correct library names are used, and follow the syntax: **%pip install PACKAGE_NAME**.

In [9]:
import rpy2.robjects as robjects
!pip install rpy2
%load_ext rpy2.ipython


## 2) Main Section for Code
### **ALL code for machine learning and dataset analysis** should be entered below.
##### Ensure that your code is clear and readable.
##### Remember to include comments and markdown notes as necessary to explain and highlight important segments of your code.

In [11]:
# from google.colab import drive
# drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [16]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


#### Remember to rename your file name to **NUS_DSESC_DATABUSTERS_XX.ipynb** and ensure that it can run successfully. Good luck and have fun!

In [18]:
%%R
dataset = read.csv('Quarterly Data.csv')


Error in file(file, "rt") : cannot open the connection


In [14]:
%%R
# load necessary libraries
library(ggplot2)
library(readr)
library(dplyr)
library(tidyr)
library(scales)

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union


Attaching package: ‘scales’



    col_factor




In [15]:
%%R
# Template code for plotting predictor against time (With economic contractions highlighted)

# The following code is for IPCONGD
# load the dataset
data <- read_csv("Quarterly Data.csv")

# remove metadata rows
data <- data[-c(1,2), ]

# convert 'sasdate' column to proper Date format
data$sasdate <- as.Date(data$sasdate, format="%m/%d/%Y")

# ensure all columns are numeric (except date)
data <- data %>% mutate(across(where(is.character), as.numeric))

# compute GDP growth as the response variable
data <- data %>% mutate(GDP_growth = (GDPC1 - lag(GDPC1)) / lag(GDPC1) * 100)

# remove first row (which contains NA from lag function)
data <- data[-1, ]

# define binary contraction indicator (1 if GDP growth is negative, 0 otherwise)
data <- data %>% mutate(Econ_Contraction = ifelse(GDP_growth < 0, 1, 0))

# identify start and end of contraction periods
data <- data %>%
  mutate(
    contraction_start = ifelse(Econ_Contraction == 1 & lag(Econ_Contraction, default = 0) == 0, sasdate, NA),
    contraction_end = ifelse(Econ_Contraction == 1 & lead(Econ_Contraction, default = 0) == 0, sasdate, NA)
  )

# ensure contraction_start and contraction_end are Dates before filling
data <- data %>%
  mutate(
    contraction_start = as.Date(contraction_start, origin = "1970-01-01"),
    contraction_end = as.Date(contraction_end, origin = "1970-01-01")
  )

# extract start and end dates for contraction periods
contraction_periods <- data %>%
  filter(!is.na(contraction_start) | !is.na(contraction_end)) %>%
  select(contraction_start, contraction_end) %>%
  tidyr::fill(contraction_start, .direction = "down") %>%
  tidyr::fill(contraction_end, .direction = "up") %>%
  filter(!is.na(contraction_start) & !is.na(contraction_end)) %>%
  distinct()

# ensure single-quarter contractions are visible
contraction_periods <- contraction_periods %>%
  mutate(
    contraction_end = ifelse(contraction_start == contraction_end, contraction_end + 30, contraction_end)
  )

# ensure contraction periods are correctly formatted as Dates AFTER fill()
contraction_periods <- contraction_periods %>%
  mutate(
    contraction_start = as.Date(contraction_start, origin = "1970-01-01"),
    contraction_end = as.Date(contraction_end, origin = "1970-01-01")
  )

# plot IPCONGD against time with economic contractions highlighted
ggplot(data, aes(x = sasdate, y = IPCONGD)) +
  # Highlight periods of economic contraction
  geom_rect(data = contraction_periods,
            aes(xmin = contraction_start, xmax = contraction_end, ymin = -Inf, ymax = Inf),
            inherit.aes = FALSE, fill = "red", alpha = 0.2) +

  # Plot IPCONGD as a line
  geom_line(color = "blue", size = 1) +

  # Fix the X-axis to show proper years
  scale_x_date(date_breaks = "5 years", date_labels = "%Y") +

  # Format Y-axis with comma separators
  scale_y_continuous(labels = scales::comma) +

  # Add title, labels, and theme
  labs(
    title = "Industrial Production: Consumer Goods Over Time (Index 2017=100)",
    subtitle = "Quarterly Data with Economic Contractions Highlighted",
    x = "Year",
    y = "Industrial Production: Consumer Goods (Index 2017=100)",
    color = "Legend"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    plot.subtitle = element_text(size = 12),
    axis.title = element_text(size = 14),
    axis.text = element_text(size = 12)
  )




Error: 'Quarterly Data.csv' does not exist in current working directory ('/content').


In [ ]:
# Forecasting the next 4 quarters with ARIMAX, as well as testing GDP Growth for stationarity
# load necessary libraries
library(readr)
library(dplyr)
library(lubridate)
library(forecast)
library(ggplot2)
library(tseries)

# load the dataset
data <- read_csv("Quarterly Data.csv")

# remove metadata rows
data <- data[-c(1,2), ]

# convert 'sasdate' column to Date format
data$sasdate <- as.Date(data$sasdate, format="%m/%d/%Y")

# ensure all columns are numeric (except 'sasdate')
data <- data %>% mutate(across(where(is.character), as.numeric))

# compute GDP growth as the response variable
data <- data %>% mutate(GDP_growth = (GDPC1 - lag(GDPC1)) / lag(GDPC1) * 100)

# remove first row (which contains NA from lag function)
data <- data[-1, ]

# convert GDP Growth to Time Series
gdp_ts <- ts(data$GDP_growth, start = c(year(min(data$sasdate)), quarter(min(data$sasdate))), frequency = 4)

# perform Augmented Dickey-Fuller (ADF) Test
adf_result <- adf.test(gdp_ts)

# print ADF Test Result
print(adf_result)

# define the selected predictors
predictors <- c("A014RE1Q156NBEA", "IPDCONGD", "IPB51110SQ", "UEMPLT5",
                "UEMP15T26", "HOUSTMW", "BAA10YM", "VIXCLSx", "AAAFFM",
                "TNWMVBSNNCBx", "CLAIMSx", "PERMITMW", "OILPRICEx", "PRFIx")

# keep only the available predictors
available_predictors <- predictors[predictors %in% colnames(data)]

# compute PRFIx_percent_change manually if PRFIx exists
if ("PRFIx" %in% available_predictors) {
  data <- data %>% mutate(PRFIx_percent_change = (PRFIx - lag(PRFIx)) / lag(PRFIx) * 100)
  available_predictors <- c(setdiff(available_predictors, "PRFIx"), "PRFIx_percent_change")
}

# remove NA values after computing percent change
data <- na.omit(data)

# prepare the dataset
X <- data %>% select(all_of(available_predictors))
y <- data$GDP_growth

# convert data to time series format
gdp_ts <- ts(y, start = c(year(min(data$sasdate)), quarter(min(data$sasdate))), frequency = 4)
X_ts <- ts(X, start = c(year(min(data$sasdate)), quarter(min(data$sasdate))), frequency = 4)

# fit an ARIMAX model (ARIMA with exogenous variables)
arimax_model <- auto.arima(gdp_ts, xreg = X_ts, seasonal = TRUE, stepwise = FALSE, approximation = FALSE)

# print model summary
summary(arimax_model)

# forecast GDP growth for Q1 2025, Q2 2025, Q3 2025, Q4 2025
future_quarters <- 4  # Forecasting up to 4 quarters ahead

# generate future dates
future_dates <- seq(max(data$sasdate) + months(3), by = "quarter", length.out = future_quarters)

# correctly structure future_X to match required rows
future_X <- data.frame(matrix(ncol = length(available_predictors), nrow = future_quarters))
colnames(future_X) <- available_predictors

# predict future values using linear regression
for (col in available_predictors) {
  model <- lm(data[[col]] ~ as.numeric(data$sasdate))  # Fit a linear model
  future_X[[col]] <- predict(model, newdata = data.frame(sasdate = as.numeric(future_dates)))[1:future_quarters]
}

# forecasting with the ARIMAX model
forecast_values <- forecast(arimax_model, xreg = as.matrix(future_X), h = future_quarters)

# extract forecasted values
forecast_gdp_growth <- data.frame(
  Quarter = c("Q1 2025", "Q2 2025", "Q3 2025", "Q4 2025"),
  Predicted_GDP_Growth = forecast_values$mean
)

# print forecasted GDP growth
print(forecast_gdp_growth)

# create forecast plot
forecast_df <- data.frame(
  Date = future_dates,
  Predicted_GDP_Growth = forecast_values$mean
)

# merge actual data and forecast
actual_df <- data.frame(Date = data$sasdate, GDP_growth = y)
plot_data <- bind_rows(actual_df %>% rename(Predicted_GDP_Growth = GDP_growth), forecast_df)

# extract forecasted values and confidence intervals
forecast_gdp_growth <- data.frame(
  Date = future_dates,
  Predicted_GDP_Growth = forecast_values$mean,
  Lower_80 = forecast_values$lower[,1],  # 80% lower bound
  Upper_80 = forecast_values$upper[,1],  # 80% upper bound
  Lower_95 = forecast_values$lower[,2],  # 95% lower bound
  Upper_95 = forecast_values$upper[,2]   # 95% upper bound
)

# merge actual data and forecast
actual_df <- data.frame(Date = data$sasdate, GDP_growth = y)
plot_data <- bind_rows(actual_df %>% rename(Predicted_GDP_Growth = GDP_growth), forecast_gdp_growth)

# plot historical and forecasted GDP growth with confidence intervals
ggplot(plot_data, aes(x = Date, y = Predicted_GDP_Growth)) +
  geom_line(color = "blue") +
  geom_point(color = "blue") +
  # Confidence interval 95% (wider range)
  geom_ribbon(data = forecast_gdp_growth, aes(ymin = Lower_95, ymax = Upper_95), fill = "gray70", alpha = 0.4) +
  # Confidence interval 80% (narrower range)
  geom_ribbon(data = forecast_gdp_growth, aes(ymin = Lower_80, ymax = Upper_80), fill = "gray50", alpha = 0.4) +
  # Forecasted values
  geom_point(data = forecast_gdp_growth, aes(x = Date, y = Predicted_GDP_Growth), color = "red", size = 3) +
  geom_line(data = forecast_gdp_growth, aes(x = Date, y = Predicted_GDP_Growth), color = "red", linetype = "dashed") +
  labs(title = "GDP Growth Forecast using ARIMAX with Confidence Intervals",
       x = "Date", y = "GDP Growth (%)",
       caption = "Red points indicate forecasted values.\nShaded areas represent 80% and 95% confidence intervals.") +
  theme_minimal()

# print forecasted GDP growth with confidence intervals
forecast_output <- data.frame(
  Quarter = c("Q1 2025", "Q2 2025", "Q3 2025", "Q4 2025"),
  Predicted_GDP_Growth = forecast_values$mean,
  Lower_80 = forecast_values$lower[,1],  # 80% confidence lower bound
  Upper_80 = forecast_values$upper[,1],  # 80% confidence upper bound
  Lower_95 = forecast_values$lower[,2],  # 95% confidence lower bound
  Upper_95 = forecast_values$upper[,2]   # 95% confidence upper bound
)

# print the results
print(forecast_output)


ERROR: Error in library(forecast): there is no package called ‘forecast’
